In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from __future__ import annotations

from pydantic import Field

from typing import Optional
import numpy as np
from qat.utils.pydref import HasId, Ref, RefDict, RefList

In [11]:
class QuantumComponent(HasId):
    pass


class PhysicalBaseband(QuantumComponent):
    pass

class PhysicalChannel(QuantumComponent):
    baseband: PhysicalBaseband


class PulseChannel(QuantumComponent):
    physical_channel: Ref[PhysicalChannel]
    #auxiliary_qubits: Optional[RefList[Qubit]] = []


class QuantumDevice(QuantumComponent):
    pulse_channels: RefDict[PulseChannel]
    physical_channel: Ref[PhysicalChannel]
    measure_device: Optional[Ref[Resonator]] = None


class Resonator(QuantumDevice):
    measure_device: None = None


class Qubit(QuantumDevice):
    measure_device: Ref[Resonator]

class LogicalQubitCoupling(HasId):
    source : Qubit
    target : Qubit
    quality : float

class PhysicalQubitCoupling(LogicalQubitCoupling):
    source_pulsechannel : Ref[PulseChannel]
    target_pulsechannel : Ref[PulseChannel]

class LogicalQubitTopology(QuantumDevice):
    couplings: list[LogicalQubitCoupling]
    

class PhysicalQubitTopology(QuantumDevice):
    couplings: list[PhysicalQubitCoupling]


In [12]:
class QuantumHardwareModel(HasId):
    """
    Base class for calibrating our QPU hardware.

    Attributes:
        qubits:
    """

    physical_basebands: RefDict[PhysicalBaseband] = Field(allow_mutation=False,
                                                        default=dict())

    physical_channels: RefDict[PhysicalChannel] = Field(allow_mutation=False, 
                                                        default=dict())

    pulse_channels: RefDict[PulseChannel] = Field(allow_mutation=False, default=dict())
    qubits: RefDict[Qubit] = Field(allow_mutation=False, default=dict())
    resonators: RefDict[Resonator] = Field(allow_mutation=False, default=dict())


This model is a DAG which means it can be serialised in layers

In [5]:
pick = lambda L, size=3: {l.id: l for l in np.random.choice(L, size=size)}

A = [At(x=i, id='A' + str(i)) for i in range(10)]
B = [Bt(x=i, id='B' + str(i), As=pick(A,3)) for i in range(10)]
C = [Ct(x=i, id='C' + str(i), As=pick(A,3), Bs=list(pick(B,3).values()), someB=list(pick(B,1).values())[0]) for i in range(10)]
D = [Dt(x=i, id='D' + str(i), Cs=pick(C,3)) for i in range(5)]

In [6]:
O1 = Outer(A=A, B=B,C=C,D=D, id='outer')
blob = O1.model_dump()

In [7]:
O2 = Outer(**blob)
assert(O2 == O1)